<a href="https://colab.research.google.com/github/HenriqueColombari/Processamento_Imagens/blob/main/Tarefa08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
import os #para trabalhar com os diretórios
import csv #salvar os resultados dos exercicios 1 e 2
import shutil
import cv2 as cv #processamento de imagens
import numpy as np #matemáticas gerais
import matplotlib.pyplot as plt #plots gráficos

from os import walk
from os import listdir #usado com o walk para retornar a lista com o nome de todos os arquivos em um diretório
from pandas import crosstab #retornar a matriz com os resultados do MLP
from google.colab import drive #comunicação com o google drive
from os.path import isfile, join
from sklearn import preprocessing #para normalização
from sklearn.neural_network import MLPClassifier #rede neural utilizada na classificação
from sklearn.model_selection import train_test_split #divisão do dataset em treino e teste

drive.mount('/content/drive')

Mounted at /content/drive


#Carregando o Dataset

In [ ]:
path = "drive//MyDrive//Processamento de Imagem//frutas_dataset//images"        #path para os arquivos carregados no drive



all_files = []                                                                  #listagem de todas as imagens dentro do diretório 
for (dirpath, dirnames, filenames) in walk(path):
  all_files.extend(filenames)

i = 0
images_load = []                                                                #lista que guaradará as imagens 
print("Quantidade de imagens: " + str(len(all_files)))
print("Carregando, isso pode demorar um pouco 😔")
while i < len(all_files):                                                       #loop para tratar e carregar todos os 300 arquivos
  temp = cv.imread(path + "//" +all_files[i], 0)

  kernel = np.ones((3,3),np.uint8) 
  temp = cv.morphologyEx(temp, cv.MORPH_OPEN, kernel, iterations = 2)           #filtragem de ruidos

  temp = cv.dilate(temp, kernel,iterations=1)                                   #operação de dilatação = preencher pequenos espaços na imagem

  temp = cv.resize(temp, (30, 30), interpolation = cv.INTER_AREA)               #redimensionar as imagens para 30x30

  for lin in range(temp.shape[0]):                                              #loop para converter em preto/branco   
    for col in range(temp.shape[1]):
    
      pixel = temp.item(lin,col)
      if ( pixel > 127):
        temp.itemset( (lin,col), 0)
      else:
        temp.itemset( (lin,col), 255)
  images_load.append([int(all_files[i][0:3]), temp])                            #adicionar a imagem a lista de imagens carregadas


  i = i + 1

images_load.sort()                                                              #ordenar a lista com as imagens

plt.figure(figsize=(15,10))
plt.subplot(121), plt.imshow(images_load[0][1],cmap='binary'), plt.axis("off")
plt.subplot(122), plt.imshow(images_load[-1][1],cmap='binary'), plt.axis("off")
plt.show()                                                                      #plot da primeira e da ultima imagem do dataset

NameError: ignored

#Exercício 1

In [ ]:
i = 0
data = []                                                                       #lista que guardará os dados que serão utilizados no MLP
while i < len(images_load):
  data.append([images_load[i][0]])                                              #guarda o indice  
  i = i + 1


i = 0
while i < len(images_load):                                                     #loop para gerar os descritores de borda de cade imagem                                        
  contours, hierarchy = cv.findContours(images_load[i][1], cv.RETR_TREE,
                                        cv.CHAIN_APPROX_NONE)
  perimetro = cv.arcLength(contours[0], True)                                   #perimetro

  diametro = np.sqrt(4*cv.contourArea(contours[0])/np.pi)                       #diametro
  
  data[i].append(perimetro)
  data[i].append(diametro)

  i = i + 1

i = 0
while i < len(data):                                                            #remover uma das colunas, caso a imagem não tenha gerado perimetro ou diametro
  if(data[i][1] == 0 or data[i][2] == 0):
    data.pop(i)
    images_load.pop(i)
    i = i - 1
  i = i + 1

print("Quanditade de imagens com dados válidos:", str(len(data)))

with open('exercicio1.csv', 'w') as myfile:                                     #gera um arquivo em formato csv para armazenar os dados gerados
    wr = csv.writer(myfile, dialect='excel')                                    #o arquivo gerado fica disponivel temporariamente nos arquivos do colab
    wr.writerow(['Index', 'Perimetro', 'Diametro'])
    i = 0
    while i < len(data):
      wr.writerow(data[i])
      i = i + 1 

Quanditade de imagens com dados válidos: 298


#Exercício 2

In [ ]:
i = 0
check_data = True
while i < len(images_load):                                                     #loop para gerar os descritores de região
  
  area = cv.countNonZero(images_load[i][1])                                     #area

  
  contours, hierarchy = cv.findContours(images_load[i][1], cv.RETR_TREE, 
                                        cv.CHAIN_APPROX_NONE)
  perimetro = cv.arcLength(contours[0], True)
  compacidade = np.square(perimetro)/area                                       # Compacidade

  
  (x,y), (eixoMenor,eixoMaior),angulo = cv.fitEllipse(contours[0])
  excentricidade = (eixoMaior/eixoMenor)                                        #excentricidade

  retangularidade = area/(eixoMaior*eixoMenor)                                  #retangularidade

  area_obj = cv.contourArea(contours[0])
  area_fechoconvexo = cv.contourArea( cv.convexHull(contours[0]))
  solidez = area_obj/area_fechoconvexo                                          #solidez
  
  if(len(data[i]) == 3):                                                        #antes de rodar o exercicio 2 novamente, deve-se executar o exercicio 1
    data[i].append(area)
    data[i].append(compacidade)
    data[i].append(excentricidade)
    data[i].append(solidez)
    data[i].append(retangularidade)                                             #append das novas features
  else:
    check_data = False

  i = i + 1


if(not check_data):
  print("Execute o exercício 1 novamente para limpar os dados")


i = 0
while i < len(data):                                                            #remover uma das colunas, caso a imagem não tenha gerado descritores de região válidos
  if(data[i][3] == 0 or data[i][4] == 0):
    data.pop(i)
    i = i - 1
  i = i + 1

print("Quanditade de imagens com dados válidos:", str(len(data)))

with open('exercicio2.csv', 'w') as myfile:                                     #gera um arquivo em formato csv para armazenar os dados gerados
    wr = csv.writer(myfile, dialect='excel')
    wr.writerow(['Index', 'Perimetro', 'Diametro', 'Area', 'Compacidade', 
                 'Excentricidade', 'Solidez', 'Retangularidade'])
    i = 0
    while i < len(data):
      wr.writerow(data[i])
      i = i + 1 




Quanditade de imagens com dados válidos: 298


#Exercicio 3

In [ ]:
temp_data = data[:]

i = 0
frutas = []
output_esperado = []
while i < len(temp_data):                                                       #loop para nomear as frutas de acordo com a classificação passada
  if(temp_data[i][0] <= 30):
    output_esperado.append('mac')
  elif(temp_data[i][0] <= 60):
    output_esperado.append('abc')
  elif(temp_data[i][0] <= 90):
    output_esperado.append('bnn')
  elif(temp_data[i][0] <= 120):
    output_esperado.append('psg')
  elif(temp_data[i][0] <= 150):
    output_esperado.append('ptg')
  elif(temp_data[i][0] <= 180):
    output_esperado.append('lrj')
  elif(temp_data[i][0] <= 210):
    output_esperado.append('mrg')
  elif(temp_data[i][0] <= 240):
    output_esperado.append('per')
  elif(temp_data[i][0] <= 270):
    output_esperado.append('lim')
  elif(temp_data[i][0] <= 300):
    output_esperado.append('uva')


  frutas.append(temp_data[i][1:])
  i = i + 1


frutas = preprocessing.normalize(frutas)                                        #normalização dos dados para passa-los no MLP


#divisao dos dados em treino e teste, 80% e 20%, respectivamente, obteve melhores resultados
frutas_train_x, frutas_test_x, frutas_train_y, frutas_test_y = train_test_split(frutas, output_esperado, test_size=0.2, random_state=42)


clf = MLPClassifier(max_iter=4000).fit(frutas_train_x, frutas_train_y)          #criação do modelo do MLP

frutas_test_y = np.array(frutas_test_y)

print("Precisão:",  str(clf.score(frutas_test_x, frutas_test_y)))               #essa previsão foi de uma boa rodada, a média foi de 75% 😃

#matriz gerada com a predicao e a classe verdadeira
print(crosstab(frutas_test_y, clf.predict(frutas_test_x), rownames=['true'], colnames=['predicted'])) 


Precisão: 0.8666666666666667
predicted  abc  bnn  lim  lrj  mac  mrg  per  psg  ptg  uva
true                                                       
abc          5    0    0    0    0    0    0    0    0    0
bnn          0    9    0    0    0    0    0    0    0    0
lim          0    0    4    0    0    0    0    0    0    0
lrj          0    0    0    4    0    2    0    0    0    0
mac          0    0    0    0    4    0    0    2    0    0
mrg          0    0    0    0    0    5    1    0    0    0
per          0    0    0    0    0    3    2    0    0    0
psg          0    0    0    0    0    0    0   10    0    0
ptg          0    0    0    0    0    0    0    0    4    0
uva          0    0    0    0    0    0    0    0    0    5
